In [3]:
import urllib
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import os, time
import shutil
import logging
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="Songs_Auto_download.log", level = logging.INFO, format=LOG_FORMAT, filemode='w')

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Get Song_SmuleRecordingsURL dictictionary

# The Following Section is a commented Code
* ==========================================================================

# Collect the Songs List
songs_list = ["Uppenantha","Karige loga","baby he loves you"]

# Song SmuleSearchurl Dictionary
smule_base_url = "https://www.smule.com/"
song_smule_searchurl = {}
for song in songs_list:
    song_smule_searchurl[song] = smule_base_url+"search?q="+song.replace(" ","%20")
song_smule_searchurl


# Get the Recording songs if available:
song_smulerecordingsurls = {}
for song, smuleurl in song_smule_searchurl.items():
    response = urlopen(smuleurl)
    data = response.read()
    response.close()
    html_page = data.decode('UTF-8')
    search_soup = soup(html_page,'html.parser')

* ==========================================================================

In [5]:
# function for Downloading the file
from requests import get
def download_file(url, file_path):
    reply = get(url, stream=True)
    with open(file_path,'wb') as file:
        for chunk in reply.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

In [6]:
# Login Function
def login(driver, url):
    user_name = "nish2203952@gmail.com"
    password = "SRINISH@vasavi5"
    driver.get(url)
    user_field = driver.find_element_by_id('auth')
    pass_field = driver.find_element_by_id('password')
    user_field.clear()
    user_field.send_keys(user_name)
    pass_field.send_keys(password)
    pass_field.send_keys(Keys.ENTER)

In [7]:
# Divide Songs List into chunks of specified size
def divide_into_chunks(bulk_list,n):
    chunked_list = []
    for i in range(0,len(bulk_list),n):
        chunked_list.append(bulk_list[i:i+n])
    return chunked_list

In [8]:
# 1.  Songs List
# "O My Brotheru","Aa ante","Thakadimithom","Feel My Love","edo priyaragam vintunna","bommani geesthe","nammaka thappani","appudo ippudo","panchadara bomma","dheera dheera dheera","arare arare","jil jil jinga","Happy Days","rooba rooba","hello rammante","chilipiga",  - > Completed Songs

logger = logging.getLogger()
logger.info("=============================  Starting Excecution  ====================================")
songs_list = ["oh priya priya","chinnadana neekosam","lachhamma","super machi","seethakalam","chal chalo chalo","mamatala talli","pacha bottasi","manohari","inka edho","my name is billa","hoyna hoyna","undiporaadhey","adiga adiga","ninnu kori","hey badhulu cheppavey","thalachi thalachi","em sandeham ledu","Vachinde","oosupodu","allasani vaari","kanulanu thaake","bekhayali","tujhe kitna chahne lage","tera ban jaunga","tum hi ho","chahun main ya naa","sunn raha hai","milne hai mujhse aayi","besabriyaan","phir kabhi","aal izz well","zoobi doobi","give me some sunshine","haan main galat","tum hi aana","thodi jagah","kadalalle","guzaarish","dangal","hanikarak bapu","tere sang yaara","Tay hai","teri mitti kesari","pachtaoge","apna time aayega","sanam re","hua hain aaj pehli baar","lag ja gale","singles anthem","samajavaragamana","butta bomma","ala vaikunthapurramuloo","ramulo ramula","pakka local","sarileru neekevvaru","mind block","lungi dance","chennai express","suryudivo chandrudivo","kuch kuch hota hai","koi mil gaya","tujhe yaad na meri aayee","ae dil hai mushkil","channa mereya","aaj jaane ki zid na karo","badtameez","dilliwali girlfriend","kabira","soch na sake"]
songs_list = list(dict.fromkeys(songs_list))  # removing Duplicates
logger.info("Removed Duplicates in Songs List")
songs_count = len(songs_list)
logger.info("No of Songs: {0}".format(songs_count))

In [13]:
logger.info("Dividing Songs into chunks - Start")
chunked_songs = divide_into_chunks(songs_list,5)
logger.info("Dividing Songs into chunks - End")
chunked_songs
for chunk in chunked_songs:
    for song in chunk:
        print(f"Song: {song}")
    print("------------------------")

Song: jil jil jinga
Song: Happy Days
Song: rooba rooba
Song: hello rammante
Song: chilipiga
------------------------
Song: oh priya priya
Song: chinnadana neekosam
Song: lachhamma
Song: super machi
Song: seethakalam
------------------------
Song: chal chalo chalo
Song: mamatala talli
Song: pacha bottasi
Song: manohari
Song: inka edho
------------------------
Song: my name is billa
Song: hoyna hoyna
Song: undiporaadhey
Song: adiga adiga
Song: ninnu kori
------------------------
Song: hey badhulu cheppavey
Song: thalachi thalachi
Song: em sandeham ledu
Song: Vachinde
Song: oosupodu
------------------------
Song: allasani vaari
Song: kanulanu thaake
Song: bekhayali
Song: tujhe kitna chahne lage
Song: tera ban jaunga
------------------------
Song: tum hi ho
Song: chahun main ya naa
Song: sunn raha hai
Song: milne hai mujhse aayi
Song: besabriyaan
------------------------
Song: phir kabhi
Song: aal izz well
Song: zoobi doobi
Song: give me some sunshine
Song: haan main galat
----------------

In [9]:
# 1.a Dividing Songs into chunks
logger.info("Dividing Songs into chunks - Start")
chunked_songs = divide_into_chunks(songs_list,5)
logger.info("Dividing Songs into chunks - End")

# 2. Song - Smule Search Urls
smule_base_url = "https://www.smule.com/"
song_smule_searchurl = {}
for chunk in chunked_songs:
    logger.info("============================ Chunk-Start =================================")
    for song in chunk:
        song_smule_searchurl[song] =  smule_base_url + "search?q=" + song.replace(" ","+")

    logger.info("Extracted 'song_smulesearch' Dictionary")

    # 3. Song-SmuleRecordingUrls
    song_smule_recordingurls = {}
    logger.info("Start: 'song_smule_recordingurls' Dictionary")
    for song, searchurl in song_smule_searchurl.items():
        logger.info("'song_smule_recordingurls', Song: '{0}' Start".format(song))
        driver = webdriver.Chrome("D:/Learnings/Self/SpringBoard-Nishanth/Main-Course/99-CapstonProject/01-HummingProject/01-DataCollection/chromedriver_win32/chromedriver.exe")
        driver.get(searchurl)

        try:
            recordings = driver.find_elements_by_xpath("//div[@class='recording-wrap']/a")
        except Exception as e:
            logger.error("Recordings for the Song '{0}' is not found in smule".format(song))
            continue

        if recordings:
            recording_urls = []
            for recording in recordings[:5]:    #limiting to first 5 recordings
                recording_urls.append(recording.get_attribute("href"))
            song_smule_recordingurls[song] = recording_urls
        driver.close()
        logger.info("'song_smule_recordingurls', Song: '{0}' End".format(song))
    logger.info("End: 'song_smule_recordingurls' Dictionary")
    logger.info("================================ song_smule_recordingurls Dictionary ============================")
    logger.info("{0}".format(song_smule_recordingurls))
    # 4. launch sing salon website and download the songs into downloads, and Move them to a different folder with song name
    logger.info("========================== Launching Sing Salon Website =================================")
    sing_salon_base_url = "https://sing.salon/smule-downloader/"
    driver = webdriver.Chrome("D:/Learnings/Self/SpringBoard-Nishanth/Main-Course/99-CapstonProject/01-HummingProject/01-DataCollection/chromedriver_win32/chromedriver.exe")

    #Login
    logger.info("Logging In with the User")
    login(driver,"https://sing.salon/login/")
    logger.info("Log In with the User Complete")

    for song, recordingurls in song_smule_recordingurls.items():
        logger.info("Downloadings-For Song: '{0}' Start".format(song))
        driver.get(sing_salon_base_url)
        path_to_watch = "C:/Users/nvpapara/Downloads/"
        projects_folder = "D:/Learnings/Self/SpringBoard-Nishanth/Main-Course/99-CapstonProject/01-HummingProject/01-DataCollection/Songs_AutoDownload/"
        before = dict([(f, None) for f in os.listdir(path_to_watch)])
        count = 1

        for url in recordingurls:
            logger.info("{0}. Recording URL:'{1}'".format(count,url))
            # enter the url in the fetch box
            # click on FetchSong Button
            fetch_box = driver.find_element_by_id("url")
            fetch_box.clear()
            fetch_box.send_keys(url)
            fetch_box.send_keys(Keys.ENTER)
            time.sleep(2)
            # find the download button
            try:
                download_button = driver.find_element_by_xpath("//div[@id='elCmsPageWrap']/a")
            except Exception as e:
                count+=1
                logger.error("Couldn't Get the Download link for Recording URL:'{0}'".format(url))
                continue

            if download_button:
                # click the download button
                song_down_link = download_button.get_attribute("href")
                if "singsalon-forum/" in song_down_link:
                    logger.error("Download link for Recording URL:'{0}' not available".format(url))
                    count +=1
                    continue
                resp = urlopen(song_down_link)
                content_type = resp.headers['content-type']
                ext = content_type[content_type.find('/')+1:]
                if not os.path.exists(projects_folder + song):
                    os.makedirs(projects_folder + song)
                to_path = projects_folder+song+"/"+ song.replace(" ","_")+ "_UserRecoding_"+ str(count)+ "." + ext
                logger.info("Recording Download Start")
                download_file(song_down_link, to_path)
                logger.info("Recording Download End")
            count += 1

        # Download Original Song from youtube to the same folder
        logger.info("Starting Youtube Download")
        yt_mp3 = "https://ytmp3.cc/en13/"
        youtube = "https://www.youtube.com/"
        driver.get(youtube + "results?search_query=" +song.replace(" ","+") + "+full+Song")
        try:
            first_link = driver.find_element_by_id('video-title').get_attribute('href')
        except Exception as e:
            logger.error("No Youtube Links Found")
            continue
        driver.get(yt_mp3)
        link_box = driver.find_element_by_id('input')
        link_box.clear()
        link_box.send_keys(first_link)
        convert_button = driver.find_element_by_id('submit')
        convert_button.click()
        time.sleep(10)
        try:
            mp3_download_link = driver.find_element_by_xpath("//div[@id='buttons']/a[text()='Download']").get_attribute('href')
        except:
            logger.error("Couldn't Get MP3 Version of the Youtube Link. Song:'{0}'".format(song))
            continue
        logger.info("MP3 Download Link:'{0}'".format(mp3_download_link))
        if not "http" in mp3_download_link:
            logger.error("Link of MP3 Version of the Youtube is empty'{0}'".format(song))
            continue
        if not os.path.exists(projects_folder + song):
            os.makedirs(projects_folder + song)
        to_path = projects_folder+song+"/"+ "You_Original-" + song.replace(" ","_")+ ".mp3"
        logger.info("Downloading the MP3(Youtube)")
        download_file(mp3_download_link, to_path)
        logger.info("Download Complete MP3(Youtube)")
        logger.info("Downloadings-For Song: '{0}' End".format(song))
    driver.close()
    logger.info("============================ Chunk-End =================================")
logger.info("Operation Successful. Check the Downloaded Songs")
logger.info("=============================  End Excecution  ====================================")
print("Operation Successful. Check the Downloaded Songs")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="url"]"}
  (Session info: chrome=81.0.4044.138)
